In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# pip install surprise
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from surprise import Reader, SVD, Dataset, accuracy
from surprise.model_selection import GridSearchCV, train_test_split, cross_validate
pd.set_option("display.max_columns", None)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Veri Setinin Hazırlanması

In [4]:
movie = pd.read_csv("/kaggle/input/movielense20m/movie.csv")
rating = pd.read_csv("/kaggle/input/movielense20m/rating.csv")
df = movie.merge(rating, how="left", on="movieId")
df.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.0,4.0,1999-12-11 13:36:47
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,6.0,5.0,1997-03-13 17:50:52
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,8.0,4.0,1996-06-05 13:37:51
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,10.0,4.0,1999-11-25 02:44:47
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,11.0,4.5,2009-01-02 01:13:41


In [5]:
movie_ids = [130219, 356, 4422, 541]
movies = ["The Dark Knight (2011)",
         "Cries and Whispers (Viskningar och rop) (1972)",
         "Forrest Gump (1994)",
         "Blade Runner (1982)"]

In [6]:
sample_df = df[df["movieId"].isin(movie_ids)]
sample_df

,movieId,title,genres,userId,rating,timestamp
2457839,356,Forrest Gump (1994),Comedy|Drama|Romance|War,4.0,4.0,1996-08-24 09:28:42
2457840,356,Forrest Gump (1994),Comedy|Drama|Romance|War,7.0,4.0,2002-01-16 19:02:55
2457841,356,Forrest Gump (1994),Comedy|Drama|Romance|War,8.0,5.0,1996-06-05 13:44:19
2457842,356,Forrest Gump (1994),Comedy|Drama|Romance|War,9.0,4.0,2001-07-01 20:26:38
2457843,356,Forrest Gump (1994),Comedy|Drama|Romance|War,10.0,3.0,1999-11-25 02:32:02
...,...,...,...,...,...,...
14742596,4422,Cries and Whispers (Viskningar och rop) (1972),Drama,137665.0,5.0,2002-03-10 01:15:59
14742597,4422,Cries and Whispers (Viskningar och rop) (1972),Drama,137851.0,4.5,2012-08-28 11:23:18
14742598,4422,Cries and Whispers (Viskningar och rop) (1972),Drama,137904.0,3.5,2007-01-18 06:01:19
14742599,4422,Cries and Whispers (Viskningar och rop) (1972),Drama,138325.0,5.0,2010-08-28 00:51:41


In [8]:
sample_df.shape

NameError: name 'df_sample' is not defined

In [9]:
user_movie_df = sample_df.pivot_table(index=["userId"], columns=["title"], values="rating")
user_movie_df.head()

title,Blade Runner (1982),Cries and Whispers (Viskningar och rop) (1972),Forrest Gump (1994),The Dark Knight (2011)
userId,,,,
1.0,4.0,NaN,NaN,NaN
2.0,5.0,NaN,NaN,NaN
3.0,5.0,NaN,NaN,NaN
4.0,NaN,NaN,4.0,NaN
7.0,NaN,NaN,4.0,NaN


In [10]:
user_movie_df.shape

(76918, 4)

In [11]:
reader = Reader(rating_scale=(1, 5))

In [12]:
data = Dataset.load_from_df(sample_df[["userId", "movieId", "rating"]], reader)

In [13]:
data

### Modelleme

In [14]:
trainset, testset = train_test_split(data, test_size=.25)

In [15]:
svd_model = SVD()

In [16]:
svd_model.fit(trainset)

In [17]:
predictions = svd_model.test(testset)

In [18]:
predictions

[Prediction(uid=107487.0, iid=356, r_ui=5.0, est=4.107592519417515, details={'was_impossible': False}),
 Prediction(uid=78691.0, iid=356, r_ui=5.0, est=4.107592519417515, details={'was_impossible': False}),
 Prediction(uid=66941.0, iid=541, r_ui=1.0, est=4.172469869355221, details={'was_impossible': False}),
 Prediction(uid=73021.0, iid=356, r_ui=5.0, est=4.107592519417515, details={'was_impossible': False}),
 Prediction(uid=80692.0, iid=356, r_ui=4.0, est=4.02649701295688, details={'was_impossible': False}),
 Prediction(uid=50160.0, iid=356, r_ui=4.0, est=4.107592519417515, details={'was_impossible': False}),
 Prediction(uid=87235.0, iid=356, r_ui=5.0, est=4.107592519417515, details={'was_impossible': False}),
 Prediction(uid=94821.0, iid=356, r_ui=5.0, est=4.107592519417515, details={'was_impossible': False}),
 Prediction(uid=44508.0, iid=541, r_ui=4.5, est=4.226960287348975, details={'was_impossible': False}),
 Prediction(uid=92796.0, iid=356, r_ui=4.0, est=4.107592519417515, detail

In [19]:
accuracy.rmse(predictions) # ortalama hata

RMSE: 0.9298


0.9297985648527082

In [20]:
# Bir kullanıcı için tahmin
svd_model.predict(uid=1.0, iid=541, verbose=True)

user: 1.0        item: 541        r_ui = None   est = 4.15   {'was_impossible': False}


Prediction(uid=1.0, iid=541, r_ui=None, est=4.1492782818377885, details={'was_impossible': False})

In [21]:
svd_model.predict(uid=1.0, iid=356, verbose=True)

user: 1.0        item: 356        r_ui = None   est = 4.41   {'was_impossible': False}


Prediction(uid=1.0, iid=356, r_ui=None, est=4.411607047875287, details={'was_impossible': False})

In [22]:
sample_df[sample_df["userId"] == 1]

,movieId,title,genres,userId,rating,timestamp
3612352,541,Blade Runner (1982),Action|Sci-Fi|Thriller,1.0,4.0,2005-04-02 23:30:03


### Model Tuning (optimization)

In [23]:
param_grid = {
    "n_epochs": [5, 10, 20],
    "lr_all": [0.002, 0.005, 0.007]}

In [24]:
gs = GridSearchCV(SVD, param_grid,
                 measures=["rmse", "mae"],
                 cv=3, n_jobs=-1, joblib_verbose=True)

In [25]:
gs.fit(data)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  27 out of  27 | elapsed:  1.3min finished


In [26]:
gs.best_score["rmse"]

0.9303427273445726

In [27]:
gs.best_params["rmse"]

{'n_epochs': 10, 'lr_all': 0.002}

### Final Model ve Tahmin

In [28]:
svd_model = SVD(**gs.best_params["rmse"])

In [29]:
data = data.build_full_trainset()
svd_model.fit(data)

In [30]:
svd_model.predict(uid=1.0, iid=541, verbose=True)

user: 1.0        item: 541        r_ui = None   est = 4.19   {'was_impossible': False}


Prediction(uid=1.0, iid=541, r_ui=None, est=4.190634934191046, details={'was_impossible': False})

In [31]:
sample_df[sample_df["userId"] == 1]

,movieId,title,genres,userId,rating,timestamp
3612352,541,Blade Runner (1982),Action|Sci-Fi|Thriller,1.0,4.0,2005-04-02 23:30:03
